<a href="https://colab.research.google.com/github/mrcruz117/SARIMAX---Parameter-Tuning-Template/blob/main/SARIMAX_Forecast_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries and Data

In [1]:
!pip install pmdarima

In [2]:
%cd /content/drive/MyDrive/Time\ Series\ Forecasting\ Product

/content/drive/MyDrive/Time Series Forecasting Product


In [3]:
#libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pmdarima as pm

In [4]:
# pmdarima dep fix
# 1. Blow away any existing installs
# !pip uninstall -y numpy

# 2. Install the NumPy version known to work (1.23.x series has ABI size 96)
# !pip install "numpy<2"

In [5]:
#load the data
#YYYY-MM-DD
df = pd.read_csv('nyc_data.csv', index_col = 0, parse_dates = True)
future_df = pd.read_csv('future.csv', index_col = 0, parse_dates = True)
future_df.head()

,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2021-01-01,NaN,0,0,0,5.00,154.221
2021-01-02,NaN,0,0,0,11.11,264.805
2021-01-03,NaN,0,0,0,3.89,115.499
2021-01-04,NaN,0,0,0,6.67,124.650
2021-01-05,NaN,0,0,0,5.56,77.968


In [6]:
#Rename variable
df = df.rename(columns = {'Demand': 'y'})
df.head(0)

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,


In [7]:
# Extract Regressors
train_X = df.iloc[:,1:]
future_X = future_df.iloc[:,1:]

future_X.head(0)

,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,


# SARIMAX Model

In [ ]:
# model

model = pm.ARIMA(
    X=X,
    order=(1,1,1),
    seasonal_order=(1,1,1, 7),
    suppress_warnings=True,
    enforce_stationarity=False,
    )